In [45]:
import lab_tools
import pandas as pd
import os 
import pickle
import numpy as np
import hvplot.pandas  # noqa
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import uncertainties as unc
import logging
import copy
import cmath
from uncertainties import unumpy
import plotly.express as px
import uncertainties.umath as umath 

In [14]:
def uplot(x_ufloats, y_ufloats):
    plt.errorbar( unumpy.nominal_values(x_ufloats), unumpy.nominal_values(y_ufloats),  xerr=unumpy.std_devs(x_ufloats), yerr= unumpy.std_devs(y_ufloats), fmt='o')  
# constants
R = unc.ufloat(19, 0.1)
N = unc.ufloat(50, 1)


In [3]:
termo_dict = lab_tools.load_pickle('termo')
# termo_dict['freq'] = 507517 # hz

In [5]:
def get_temperature_from_file_names(d):
    temperatures = []

    for x in d.keys():
        temperatures.append(-float(x))
    return temperatures


amplitudes = lab_tools.find_amp(termo_dict)
temps = get_temperature_from_file_names(termo_dict)

# damy_amp = np.ones(amplitudes.shape)
p = termo_dict['97']

In [11]:
def find_zeros(x, y):
    return x[y == 0]

def find_phase_diff(df, w):
    p_0 = lab_tools.findzeros(df['t'], df['x'])[0][0]
    p_1 = lab_tools.findzeros(df['t'], df['y'])[0][0]
    # p_0 = find_zeros(np.array(df['t']), np.array(df['x']))[0]
    # p_1 = find_zeros(np.array(df['t']), np.array(df['y']))[0]

    delta_p = w * (p_1 - p_0)
    return delta_p

phases = []

for k,df in termo_dict.items():
    val = find_phase_diff(df, lab_tools.Constants.Termo.f_in)
    print(val)
    phases.append(val)

0.6597715823326593
0.619170653722109
0.10150324774489998
0.15225441485204852
0.09135301939863984
0.09135301939863984
0.10150324774489998
0.09135301939863984
0.11165348116633003
0.08120278597720978
0.13195394800919014
0.21315673398639992
0.10150324774489998
0.6800720491755194
2.740591434587759
3.2278081606013593
0.7308241348884391
0.7003734396993188
2.720290957594559
0.7105236731207488
0.11165348116633003
0.12180371458776008
0.20300650056496988
0.6496213539863992
0.09135301939863984
0.09135301939863984
0.09135301939863984
0.12180371458776008
0.09135301939863984


In [ ]:
amplitudes

In [ ]:
import logging
logging.basicConfig(level=logging.WARNING)

phases = []
frequencies = []
amplitudes = []
faild_fits = {}

def find_phase(w,a1,p1,c1,a2,p2,c2):
    f = 2 * np.pi / w
    return f * np.abs(p1 - p2)

def find_z(w,a1,p1,c1,a2,p2,c2):
    return a2 / a1

funcs = {'phase': find_phase, 'z': find_z}
limits = {'z': [0, 10000]}


for freq, df in copy.deepcopy(termo_dict).items():
    float_freq = float(freq)
    logging.info(f"frequency is {freq}")
    result = lab_tools.extract_data_from_fit(funcs, df, lab_tools.Constants.Termo.f_in, display=False)

    if result:
        frequencies.append(float(freq))
        phases.append(result['phase'])
        amplitudes.append(result['z'])

    else:
        faild_fits[freq] = df
        del termo_dict[freq]

In [67]:
from scipy.constants import mu_0 as mu_0
import math

def calc_R_effective(amplitude, phase):
    return amplitude*np.cos(phase) - lab_tools.Constants.R

def calc_L_effective(amplitude, phase):
    return amplitude*np.sin(phase)/(2*np.pi)

def calc_mu(amplitude, phase, N, freq):
    l_effective = calc_L_effective(amplitude, phase)
    r_effective = calc_R_effective(amplitude, phase)

    real_mu = np.abs(r_effective/(mu_0*freq*N**2))
    im_mu = l_effective/(mu_0*freq*(N**2))

    # complex_mu = complex(real_mu, im_mu)

    return real_mu, im_mu, umath.sqrt(real_mu**2 + im_mu**2)

mu_df = pd.DataFrame(columns=['temp', 'r', 'i', 'abs'])
mu_df_with_errors = pd.DataFrame(columns=['temp', 'r', 'i', 'abs'])
temps = get_temperature_from_file_names(termo_dict)

for i in range(len(amplitudes)):
    r_, i_, abs_ = calc_mu(amplitudes[i], phases[i], lab_tools.Constants.N, lab_tools.Constants.Termo.f_in)
    mu_df_with_errors.loc[i] = [temps[i], r_, i_, abs_]
    mu_df.loc[i] = [unumpy.nominal_values(temps[i]), unumpy.nominal_values(r_), unumpy.nominal_values(i_), unumpy.nominal_values(abs_)]

def uplot(x_ufloats, y_ufloats):
    plt.errorbar( unumpy.nominal_values(x_ufloats), unumpy.nominal_values(y_ufloats),
      xerr=unumpy.std_devs(x_ufloats), yerr= unumpy.std_devs(y_ufloats), fmt='o')  

def pplot(x_ufloats, y_ufloats):
    fig = px.scatter(unumpy.nominal_values(x_ufloats), unumpy.nominal_values(y_ufloats),
      error_x=unumpy.std_devs(x_ufloats), error_y= unumpy.std_devs(y_ufloats))
    fig.show()


amp_phase_df = pd.DataFrame(columns=['temps', 'amps', 'phases'])
amp_phase_df['temps'] = temps
amp_phase_df['phases'] = phases
amp_phase_df['amps'] = unumpy.nominal_values(amplitudes)

# amplitudes = damy_amp
# R_effective = []
# L_effective = []

# for index in range(len(amplitudes)):
#     R_effective.append(amplitudes[index]*np.cos(phases[index]) - lab_tools.Constants.R)
#     L_effective.append(amplitudes[index]*np.sin(phases[index])/(2*np.pi))

# Real_mu = []
# Im_mu = []

# for index in range(len(R_effective)):
#     Real_mu.append(np.abs(R_effective[index]/(mu_0*frequencies[index]*(lab_tools.Constants.N**2))))
#     Im_mu.append(L_effective[index]/(mu_0*frequencies[index]*(N**2)))



In [74]:
mu_df
# lab_tools.uplot(mu_df['temp'], mu_df['r'])
# pplot(mu_df['temp'], mu_df['i'])
# mu_df.hvplot.scatter(x='temp')
mu_df.hvplot.scatter(x='temp', y='i')
# amp_phase_df.hvplot.scatter(x='temps', y='phases')
# amp_phase_df

:Scatter   [temp]   (i)

In [13]:
path ="/Users/user/Documents/semster_c/courses/lab/magnetisem/extension2/termo/balanced"
d = lab_tools.read_to_dict(path)
termo_df = lab_tools.find_amp(d)
termo_df = termo_df.drop(4)
termo_df = termo_df.drop(12)
termo_df

temperature  amplitude
0         -97.0   6.650000
1         -90.5   9.975000
2         -61.8  47.879999
3         -94.0   7.600000
5         -76.5  19.949999
6         -56.5  60.799999
7         -47.0  79.799998
8         -94.5  21.763636
9         -39.9  79.799998
10        -70.5  27.022222
11        -87.0  10.133333
13       -121.0   5.093617
14        -79.8  16.510344
15       -124.0   6.138462
16       -130.0   5.320000
17       -127.0   5.093617
18       -134.5   5.093617
19       -100.0   6.300000
20       -115.0   5.174468
21       -139.0   4.987500
22       -103.8  13.300000
23        -86.5  11.400000
24       -107.0   6.138462
25       -113.0   7.152941
26        -75.0  27.022222
27        -48.0  68.399998
28        -62.0  48.639999
29        -83.5  13.511111
30        -59.0  47.879999

In [24]:
find_phase_diff(p)

([2.36800006e-05, 0.0], [-1.0399995e-06, 0.0])
([2.497999958e-05, 0.0], [-1.99995203e-08, 0.0])


1.2999989799999987e-06

In [38]:
d = find_zeros(np.array(df['t']), np.array(df['x']))
d

array([-2.47600001e-05, -2.27799992e-05, -2.08000001e-05, -1.97999998e-05,
       -1.78199989e-05, -1.58399998e-05, -1.28999991e-05, -1.09199991e-05,
       -7.93999971e-06, -5.95999927e-06, -3.01999944e-06, -1.03999950e-06,
        1.94000040e-06,  3.92000038e-06,  8.84000019e-06,  1.08200002e-05,
        1.18200005e-05,  1.38000005e-05,  1.57800005e-05,  1.87200003e-05,
        2.07000012e-05,  2.36800006e-05])

In [25]:
p = termo_dict['97']
p.hvplot.scatter(x='t')


:NdOverlay   [Variable]
   :Scatter   [t]   (value)